In [1]:
import torchvision
import torch
from torchvision import transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt

import pyro
import pyro.contrib.gp as gp
import pyro.distributions as dist
from models.networks import *

In [2]:
feature_net = Simple()
feature_net.load_state_dict(torch.load("cnn_75.pt"))

<All keys matched successfully>

In [36]:
transformations = transforms.Compose([
    transforms.CenterCrop(255),
    transforms.Resize(100),
    transforms.ToTensor()])

voc_data = torchvision.datasets.Cityscapes('data', split="test", mode="fine", target_type="semantic",
                                           transform=transformations, target_transform=transformations)
data_loader = torch.utils.data.DataLoader(voc_data,batch_size=32,shuffle=True)

In [4]:
def get_features(image, seg, gt):
    feats = torch.squeeze(feature_net(torch.unsqueeze(image, dim=0), torch.unsqueeze(seg.view(-1, 100, 100), dim=0)).permute(0, 2, 3, 1))
    x = feats.detach()
    y = gt.detach()/gt.max()
    return x, y

In [5]:
def get_least_confident(X, y, cov, samples_seen):
    lcs = np.argsort(cov.detach().numpy())
    for lc_i in lcs:
        i = lc_i // 100
        j = lc_i % 100
        if (i, j) not in samples_seen:
            break
    return (i,j)

In [ ]:
from sklearn.model_selection import train_test_split

In [34]:
input_dim = 75
num_samples = 50
num_steps = 10
total = 0.0
samples_seen = []
cur_segs = []
for i in [1]:
    one, two = voc_data.__getitem__(i)
    cur_seg = torch.zeros((100, 100))
    X, y = get_features(one, cur_seg, two)
    first_sample = (0, 0)
    samples_seen.append(first_sample)
    X_train = np.array([X[sample].numpy() for sample in samples_seen])
    y_train = np.array([y[0, sample[0], sample[1]].numpy() for sample in samples_seen])
    kernel = gp.kernels.RBF(input_dim=input_dim, variance=torch.tensor(1.),
                            lengthscale=torch.tensor(10.))
    gpr = gp.models.GPRegression(torch.from_numpy(X_train).float(), 
                                 torch.from_numpy(y_train).float(), kernel, noise=torch.tensor(1.))
    optimizer = torch.optim.Adam(gpr.parameters(), lr=0.005)
    loss_fn = pyro.infer.Trace_ELBO().differentiable_loss
    losses = []
    for i in range(num_samples):
        gpr.set_data(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float())
        for j in range(num_steps):
            optimizer.zero_grad()
            loss = loss_fn(gpr.model, gpr.guide)
            loss.backward()
            optimizer.step()
        losses.append(loss.item())
        #print(j, loss.item())
        mean, cov = gpr(X.view(-1, input_dim))
        next_x = get_least_confident(X, y, cov.detach().float(), samples_seen)
        #print(next_x)
        samples_seen.append(next_x)
        cur_segs.append(cur_seg.detach().numpy())
        cur_seg[next_x[0], next_x[1]] = y[0, next_x[0], next_x[1]]
        X, y = get_features(one, cur_seg, two)
        X_train = np.array([X[sample].numpy() for sample in samples_seen])
        y_train = np.array([y[0, sample[0], sample[1]].numpy() for sample in samples_seen])
        #print(np.nonzero(cur_seg.detach().numpy()))
    loss = np.mean((mean.view(-1, 100, 100).detach().numpy()-y.numpy())**2)
    total += loss
print(total/(i+1))

0.0008677465468645096


In [ ]:
plt.imshow(y.reshape(100, 100))

In [ ]:
plt.imshow(cur_seg.reshape(100, 100))

In [ ]:
plt.imshow(mean.detach().numpy().reshape(100, 100))